In [3]:
# `setup.jl` will load core packages including the ED package
include("./src/setup.jl")

print_symbols (generic function with 1 method)

In [4]:
print_symbols()

id, σx, σy, σz, ⊗(x,y)


Other sources:
- https://arxiv.org/pdf/cond-mat/0010440.pdf

# Majorana intro

This notebook walks through the process of operating in the majorana basis. Majorana particles are defined as particles that are their own antiparticle. In the language of second quantization, this is simply the statement that creating the particle, and annihilating it should be done with the same operator.

I'll open with a quick Majorana intro, considering the Hamiltonian: $H=-\frac{1}{2m} \frac{\partial^2}{\partial x^2} - V \delta(x)$ with associated raising and lowering operatrs $c^\dagger$ and $c$ respectively.

We can fabricate operators $\gamma_+=c+c^\dagger$, $\gamma_-=-i(c-c^\dagger)$ that satisfy the majorana condition $\gamma_\pm = \gamma_pm^\dagger$, each of the operators square to one, and you can construct a number operator from them $n = c^\dagger c = 0.5(i \gamma_+ \gamma_- + 1)$.

https://www.theorie.physik.uni-muenchen.de/activities/schools/archiv/asc_school_15/refael_notes.pdf

# Jordan Wigner Transformation

All of this in the context of the Ising model's Hamiltonian:

$$ H = -J g \sum_i \hat{\sigma}_i^x - J \sum_{<i,j>} \hat{\sigma}_i^z \hat{\sigma}_j^z $$

Starting out defining raising and lowering operators for the Ising chain:

$$ \hat{\sigma}_i^\pm = \frac{1}{2} (\hat{\sigma}_i^x \pm i \hat{\sigma}_i^y) $$

Which satisfies the anti-commutation relations: $\{ \sigma_i^-, \sigma_i^+ \} = 1$, $\{\sigma_i^-, \sigma_i^-\} = \{\sigma_i^+, \sigma_i^+\} = 0 $

And just for fun we can construct the number operator $n_l = c_l^\dagger c_l$.

The Jordan-Wigner transformation can be applied, recovering the true fermion anticommutation relations from the spin operators. (otherwise, interpretting the raising and lowering operators as creation and anhiliation operators are interpretted as being bosonic as they commute at different sites $[\sigma_i^+,\sigma_j^-] = 0, j \neq i$, while fermionic operators anticommute at different sites). 

This Jordan-Wigner transformation is as follows:

$$ c_i = \Pi_{j<i} (\hat{\sigma}_j^z) \hat{\sigma}_i^+ $$
$$ c_i^\dagger = \Pi_{j<i} (\hat{\sigma}_j^z) \hat{\sigma}_i^- $$

These fermionic operators are non-local. The spin operator is only defined at a point (local), whie the fermionic operator depends on the spin values along a whole line, from the left boundary to the given location.

The Jordan-Wigner transformation is also reversible via:

$$ \hat{\sigma}_i^+ = \Pi_{j<i} (1 - 2 c_j^\dagger c_j) c_i $$
$$ \hat{\sigma}_i^- = \Pi_{j<i} (1 - 2 c_j^\dagger c_j) c_i^\dagger $$

# Kitaev Chain

We are working with the Hamiltonian for the Kitaev chain, written below. It is a simplification of the hamiltonian used throughout the codebase in which the parameter $J_z=0, \gamma \neq 0$.

$$ H = H_{xx} + H_{yy} + H_z $$

$$ H = \sum\limits_{i}^{L} [J (\frac{1+\gamma}{2})σ^{x}_{i}σ^{x}_{i+1} + J (\frac{1-\gamma}{2})σ^{y}_{i}σ^{y}_{i+1} + g σ^{z}_{i}] $$


The same Hamiltonian written in the Majorana basis can be constructed after a Jordan-Wigner transformation.

Map the problem to Majorana fermions:

$$ a_{2l-1} = \Pi_{j=1}^{l-1} \sigma_j^z \sigma_l^x $$

$$ a_{2l} = \Pi_{j=1}^{l-1} \sigma_j^z \sigma_l^y $$

where l = 1, 2, ..., L.

Now say $$\vec{a} = (a_1, a_2, ..., a_{2L})$$


In [15]:
ED.Psi0(2,0.5)

4×4 Matrix{Float64}:
 0.0        0.894427  0.447214   0.0
 0.894427   0.0       0.0       -0.447214
 0.447214   0.0       0.0        0.894427
 0.0       -0.447214  0.894427   0.0

In [5]:
function get_σix(L::Int64, i::Int64)
    ops = fill(id, L)
    ops[end-i+1] = σx
    return foldl(⊗, ops)
end

function get_σiy(L::Int64, i::Int64)
    ops = fill(id, L)
    ops[end-i+1] = σy
    return foldl(⊗, ops)
end

function get_σiz(L::Int64, i::Int64)
    ops = fill(id, L)
    ops[end-i+1] = σz
    return foldl(⊗, ops)
end

get_σiz (generic function with 1 method)

In [14]:
function get_U(L::Int64, jx::Float64, g::Float64, gamma::Float64, T::Float64)
    return exp.( -im * T/2 * 
            ( ED.Hxx(L, jx*(1+gamma)/2) + 
            ED.Hyy(L, jx*(1-gamma)/2) + 
            g * ED.Hz(L, jx*(1+gamma)/2) ) 
        )
end

function get_majorana_a_l(L::Int64, l::Int64)
    """
    Returns the lth majorana state
    """
    if l%2==0 # l even
        a = get_σiz(1, L) * get_σix(1, l)
        for j in 2:l-1
            a = a * ( get_σiz(L, j) * get_σix(l, 1) )
        end
    else # l odd
        a = get_σiz(1,L) * get_σix(1, l)
        for j in 2:l-1
            a = a * ( get_σiz(L, j) * get_σix(l,1) )
        end
    end
    
    return a
    
end

# function get_majorana_vector_a(L::Int64)
#     a_vector = zeros(L)
#     for i in 1:L
#         a_vector[i] = get_majorana_a_l(L, i)
#     end
#     return a_vector
# end

get_majorana_a_l (generic function with 1 method)

In [9]:
get_majorana_a_l(4, 2)

LoadError: BoundsError: attempt to access 1-element Vector{Matrix{Complex{Int64}}} at index [-2]

In [130]:
get_U( 4, 1.0, 1.0, 3.14)

16×16 Matrix{ComplexF64}:
    0.999995+0.0031853im  …          1.0-0.0im
         1.0-0.0im                   1.0-0.0im
         1.0-0.0im                   1.0-0.0im
 0.000796327-1.0im           0.000796327-1.0im
         1.0-0.0im                   1.0-0.0im
         1.0-0.0im        …          1.0-0.0im
 0.000796327-1.0im                   1.0-0.0im
         1.0-0.0im                   1.0-0.0im
         1.0-0.0im                   1.0-0.0im
         1.0-0.0im           0.000796327-1.0im
         1.0-0.0im        …          1.0-0.0im
         1.0-0.0im                   1.0-0.0im
 0.000796327-1.0im           0.000796327-1.0im
         1.0-0.0im                   1.0-0.0im
         1.0-0.0im                   1.0-0.0im
         1.0-0.0im        …     0.999995-0.0031853im

In [83]:
typeof(σy)

Matrix{Complex{Int64}} (alias for Array{Complex{Int64}, 2})

In [108]:
typeof( convert(Matrix{Complex{Int64}}, σy) )

Matrix{Complex{Int64}} (alias for Array{Complex{Int64}, 2})

In [121]:
test = Matrix{Complex{Int64}}[]
append!(test, convert(Matrix{Complex{Int64}}, σy))
# append!(test, id)

LoadError: MethodError: [0mCannot `convert` an object of type [92mComplex{Int64}[39m[0m to an object of type [91mMatrix{Complex{Int64}}[39m
[0mClosest candidates are:
[0m  convert(::Type{Array{T, N}}, [91m::StaticArrays.SizedArray{S, T, N, N, Array{T, N}}[39m) where {S, T, N} at /Users/newberry/.julia/packages/StaticArrays/aMe0r/src/SizedArray.jl:120
[0m  convert(::Type{Array{T, N}}, [91m::StaticArrays.SizedArray{S, T, N, M, TData} where {M, TData<:AbstractArray{T, M}}[39m) where {T, S, N} at /Users/newberry/.julia/packages/StaticArrays/aMe0r/src/SizedArray.jl:114
[0m  convert(::Type{T}, [91m::AbstractArray[39m) where T<:Array at array.jl:532
[0m  ...

In [85]:
test = Complex{Int64}[]
append!(test, σy)
# append!(test, id)

4-element Vector{Complex{Int64}}:
 0 + 0im
 0 + 1im
 0 - 1im
 0 + 0im

In [112]:
test = Matrix[]

Matrix{T} where T[]

In [120]:
append!(test, [[1,2,3] [1,2,3]])

LoadError: MethodError: [0mCannot `convert` an object of type 
[0m  [92mInt64[39m[0m to an object of type 
[0m  [91mMatrix{T} where T[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::AbstractArray[39m) where T<:Array at array.jl:532
[0m  convert(::Type{T}, [91m::Factorization[39m) where T<:AbstractArray at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/factorization.jl:58
[0m  convert(::Type{T}, [91m::T[39m) where T<:AbstractArray at abstractarray.jl:14
[0m  ...

In [ ]:
test[1]

In [25]:
#Psi0(L,g)
A = ED.Psi0(4, 0.5)
evals, evecs = eigen(A)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
16-element Vector{Float64}:
 -0.9999999999999999
 -0.9999999999999991
 -0.9999999999999991
 -0.9999999999999991
 -0.9999999999999991
 -0.9999999999999989
 -0.9999999999999987
 -0.9999999999999987
  0.9999999999999999
  0.9999999999999999
  0.9999999999999999
  0.9999999999999999
  1.0
  1.0
  1.0000000000000002
  1.0000000000000007
vectors:
16×16 Matrix{Float64}:
  0.0718496     0.445932      0.543654     …   0.00419344   -0.0549316
 -0.0739935    -0.132711     -0.683182         0.0272224    -0.0071049
 -0.0909302    -0.60003       0.108079        -0.0296093    -0.110612
  0.0165755     0.491912     -0.406604        -0.0789999    -0.133169
 -0.182423     -0.258941      0.0463206        0.19209      -0.231517
  0.175919      0.184099     -0.184032     …   0.11957      -0.23401
 -0.014925      0.0221277    -0.0158098        0.120936     -0.0543207
  0.0856793    -0.0460378     0.0220793        0.0359245     0.0255041
  0.6

In [ ]:
for ti in 1:length(times)
    print("ed: ti = $ti\r")
    U = Diagonal(exp.(-im*times[ti].*vals))
    ainfs[ti] = 1/(2^L)*tr(U'*sigxV*U*sigxV)
end

In [41]:
Ls = [6,8]
gs = range(-.4,.4,length = 41)
jx = 1.0
tol = 1e-13

for (Li,L) in enumerate(Ls), (gi,g) in enumerate(gs)

    ham = ED.H(L,jx,0.0,g)
    psi0 = ED.Psi0(L,g)
    # @printf("g^L = %.2e, max of comm = %.2e ",g^L,maximum(abs.(ham*psi0 - psi0*ham)))
    tmp = psi0*psi0
#     println(tmp)
    tmp2 = maximum(Diagonal(tmp))
#     @test abs(tmp2 - 1.0) < tol
end


In [43]:
g = 0.5
L=4
ham = ED.H(L,1.0,0.0,g)
psi0 = ED.Psi0(L,g)
# @printf("g^L = %.2e, max of comm = %.2e ",g^L,maximum(abs.(ham*psi0 - psi0*ham)))
tmp = psi0*psi0

16×16 Matrix{Float64}:
  1.0           0.0           0.0          …   0.0           0.0
  0.0           1.0           2.31066e-17      0.0           0.0
  0.0           2.31066e-17   1.0             -1.44416e-18   0.0
 -2.31066e-17   0.0           0.0              0.0          -1.44416e-18
  0.0           1.15533e-17   5.77664e-18      2.88832e-18   0.0
 -1.15533e-17   0.0           0.0          …   0.0           2.88832e-18
 -5.77664e-18   0.0           0.0              0.0          -5.77664e-18
  0.0          -5.77664e-18   1.15533e-17      5.77664e-18   0.0
  0.0           5.77664e-18   2.88832e-18     -5.77664e-18   0.0
 -5.77664e-18   0.0           0.0              0.0          -5.77664e-18
 -2.88832e-18   0.0           0.0          …   0.0           1.15533e-17
  0.0          -2.88832e-18   5.77664e-18     -1.15533e-17   0.0
 -1.44416e-18   0.0           0.0              0.0          -2.31066e-17
  0.0          -1.44416e-18   0.0              2.31066e-17   0.0
  0.0           0.0

In [44]:
tmp

16×16 Matrix{Float64}:
  1.0           0.0           0.0          …   0.0           0.0
  0.0           1.0           2.31066e-17      0.0           0.0
  0.0           2.31066e-17   1.0             -1.44416e-18   0.0
 -2.31066e-17   0.0           0.0              0.0          -1.44416e-18
  0.0           1.15533e-17   5.77664e-18      2.88832e-18   0.0
 -1.15533e-17   0.0           0.0          …   0.0           2.88832e-18
 -5.77664e-18   0.0           0.0              0.0          -5.77664e-18
  0.0          -5.77664e-18   1.15533e-17      5.77664e-18   0.0
  0.0           5.77664e-18   2.88832e-18     -5.77664e-18   0.0
 -5.77664e-18   0.0           0.0              0.0          -5.77664e-18
 -2.88832e-18   0.0           0.0          …   0.0           1.15533e-17
  0.0          -2.88832e-18   5.77664e-18     -1.15533e-17   0.0
 -1.44416e-18   0.0           0.0              0.0          -2.31066e-17
  0.0          -1.44416e-18   0.0              2.31066e-17   0.0
  0.0           0.0

In [ ]:
"""
Binary `BitArray` representation of the given integer `num`, padded to length `N`.
"""
bit_rep(num::Integer, N::Integer) = BitArray(parse(Bool, i) for i in string(num, base=2, pad=N))

"""
    generate_basis(N::Integer) -> basis

Generates a basis (`Vector{BitArray}`) spanning the Hilbert space of `N` spins.
"""
function generate_basis(N::Integer)
    nstates = 2^N
    basis = Vector{BitArray{1}}(undef, nstates)
    for i in 0:nstates-1
        basis[i+1] = bit_rep(i, N)
    end
    return basis
end

function GetHamiltonian(N::Int64,g::Float64,𝛾::Float64,J::Float64,Jz::Float64)
    if N>12
        return nothing
    end
    
    # vector of operators: [σx, σx, id, ...]
    first_term_ops = fill(id, N)
    first_term_ops[1] = σx
    first_term_ops[2] = σx
    
    # vector of operators: [σy, σy, id, ...]
    second_term_ops = fill(id, N)
    second_term_ops[1] = σy
    second_term_ops[2] = σy
    
    # vector of operators: [σz, σz, id, ...]
    third_term_ops = fill(id, N)
    third_term_ops[1] = σz
    third_term_ops[2] = σz
    
    # vector of operators: [σz, id, ...]
    fourth_term_ops = fill(id, N)
    fourth_term_ops[1] = σz
    
    H = zeros(Int, 2^N, 2^N)
    # Term 1
    for i in 1:N-1
        # tensor multiply all operators
        H -= J*((1+𝛾)/2) * foldl(⊗, first_term_ops)
        # cyclic shift the operators
        first_term_ops = circshift(first_term_ops,1)
    end
        
    # Term 2
    for i in 1:N-1
        # tensor multiply all operators
        H -= J*((1-𝛾)/2) * foldl(⊗, second_term_ops)
        # cyclic shift the operators
        second_term_ops = circshift(second_term_ops,1)
    end
        
    # Term 3
    for i in 1:N-1
        # tensor multiply all operators
        H -= Jz * foldl(⊗, third_term_ops)
        # cyclic shift the operators
        third_term_ops = circshift(third_term_ops,1)
    end
        
    # Term 4 (only non-interaction term)
    for i in 1:N
        H -= g * foldl(⊗, fourth_term_ops)
        fourth_term_ops = circshift(fourth_term_ops,1)
    end
    
    return H
end

function get_σix(i::Int64, N::Int64)
    ops = fill(id, N)
    #ops[i] = σx
    ops[end-i+1] = σx
    return foldl(⊗, ops)
end

function get_Ainf_AN(L::Int64,g::Float64,jx::Float64,jz::Float64, 
        gamma::Float64,times::AbstractArray)
    #=
    Partially optimized. Refactored from Daniel Yates' code
    =#
    H = GetHamiltonian(L,g,gamma,jx,jz)
    
    ainfs = zeros(Complex{Float64}, length(times))
    
    sigx = get_σix(1, L)
    
    vals, vecs = eigen(H)
    
    H_d = zeros( size(H) )
    for i in 1:size(vals)[1]
        H_d[i,i] = vals[i]
    end
    
    sigxV = vecs'*sigx*vecs
    for ti in 1:length(times)
        print("ed: ti = $ti\r")
        U = Diagonal(exp.(-im*times[ti].*vals))
        ainfs[ti] = 1/(2^L)*tr(U'*sigxV*U*sigxV)
    end
    return real.(ainfs)
end